## Transfer Learning in TensorFlow : Feature Extraction  

Transfer learning is a machine learning method where a model developed for a task is reused as the starting point for a model on a second task.

It is a popular approach in deep learning where pre-trained models are used as the starting point on computer vision and natural language processing tasks given the vast compute and time resources required to develop neural network models on these problems and from the huge jumps in skill that they provide on related problems.

Concepts covered in this Notebook:  
* Introduce transfer learning with TensorFlow
* Using a small dataset to experiment faster(10% of training samples)
* Building a transfer learning feature extraction model with TensorFlow Hub.
* Use TensorBoard to track modelling experiments and results

**Why use transfer learning?**  

* Can leverage an existing neural network architecture **proven to work** on problems similar to our own
* Can leverage a working network architecture which has **already learned patterns** on similar data to our own (often results in great ML products with less data)


Example of tranfer learning use cases:  
* Computer Vision (using ImageNet for our Image classification problems)
* Natural Language Processing (detecting spam mails - spam filter)

### Downloading and getting familiar with data


In [ ]:
# Get data (10% of 10 food classes from food101 dataset from kaggle)
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()


--2022-02-18 05:04:13--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.107.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   210MB/s    in 0.8s    

2022-02-18 05:04:14 (210 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [ ]:
# How many images in each folder
import os
 
# Walk through 10 percent data dir and list number of files
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent)"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

### Creating data loader (preparing the data)  

We'll use the `ImageDataGenerator` class to load in our images in batches.


In [ ]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train"
test_dir = "10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size = IMAGE_SHAPE,
                                                          batch_size = BATCH_SIZE,
                                                          class_mode = "categorical")

print("Testing Images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = IMAGE_SHAPE,
                                             batch_size = 32,
                                             class_mode ="categorical")

Training images:
Found 750 images belonging to 10 classes.
Testing Images:
Found 2500 images belonging to 10 classes.


### Setting up callbacks (things to run whilst our model trains)

Callbacks are extra functionality you can add to your models to be performed during or after training. Some of the most popular callbacks:  

* Tracking experiments with the `TensorBoard` callback  
* Model checkpoint with the `ModelCheckpoint` callback
* Stopping a model from training (before it trains too long and overfits) with the `EarlyStopping` callback  

Some popular callbacks include:    

| Callback name | Use case | Code |
| --- | --- | --- |
| TensorBoard | Log the performance of multiple models and then view and compare these models in a visual way on TensorBoard (a dashboard for inspecting neural network parameters). Helpful to compare the results of different models on your data | `tf.keras.callbacks.TensorBoard()` |
| Model checkpointing | Save your model as it trains so you can stop training if needed and come back to continue off where you left. Helpful if training takes a long time and can't be done in one sitting | `tf.keras.callbacks.ModelCheckpoint()` |
| Early Stopping | Leave your model training for an arbitary amount of time and have it stop training automatically when it ceases to improve. Helpful when you've got a large dataset and don't know how long training will take | `tf.keras.callbacks.EarlyStopping()`

In [ ]:
# Create TensorBoard callback (functionized because we need to create a new one for each model)
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f"Saving TensorBoard log files to : {log_dir}")
  return tensorboard_callback


> **Note:**  You can customize the directory where your TensorBoard logs (model training metrics) get saved to whatever you like. The `log_dir` parameter we've created above is only one option.

### Creating models using Tensorflow Hub  

TensorFlow Hub is the repository of pre-trained machine learning models created with tensorflow.  

We are going to do a similar process, except the majority of our model's layers are going to come from TensorFlow Hub.

[TensorFlow Hub](https://www.tensorflow.org/hub)

In [ ]:
# Let's compare the following two models
efficientnet_url = "https://tfhub.dev/google/efficientnet/b0/classification/1"

resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

In [ ]:
# Import dependencies
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Let's make a create_model() function to  create a model from a URL
def create_model(model_url, num_classes = 10):
  """
  Takes a TensorFlow Hub URL and creates a Keras Sequential model with it

  Args:
    model_url(str): A TensorFlow Hub features extraction URL.
    num_classes(int) : Number of output neurons in the output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature extractor
    layers and Dense output layers with num_classes output neurons.
  """
  # Download the pretrained model and save it 
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable =False,
                                           name = "feature_extraction_layer",
                                           input_shape = IMAGE_SHAPE+(3,)) # Freeze the already learned parameters

  # Create our model
  model = tf.keras.Sequential([
     feature_extractor_layer,
     layers.Dense(num_classes, activation = "softmax", name = "output_layer")                               
  ])      

  return model                                  
  

### Creating and testing ResNet TensorFlow Hub Feature Extraction model

In [ ]:
# Create Resnet model
resnet_model = create_model(resnet_url,
                            num_classes=train_data_10_percent.num_classes)

In [ ]:
resnet_model.compile(loss = "categorical_crossentropy",
                     optimizer = tf.keras.optimizers.Adam(),
                     metrics = ["accuracy"])

In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction_layer (K  (None, 2048)             23564800  
 erasLayer)                                                      
                                                                 
 output_layer (Dense)        (None, 10)                20490     
                                                                 
Total params: 23,585,290
Trainable params: 20,490
Non-trainable params: 23,564,800
_________________________________________________________________


whoaa..the number of parameters are ~23 Million. But only ~20,000 are trainable parameters other are from the freeze model we uploaded from the tensorflow hub. Here's look at the Resnet50 architecture:  

![Resnet50 V2 architecture](https://miro.medium.com/max/1024/1*BnoNVpj7uCNMOFOj1DQBQA.png "Resnet50 V2 architecture").


In [ ]:
# Fit the resnet model to our 10% of the data 
resnet_history = resnet_model.fit(train_data_10_percent,
                 epochs = 5,
                 steps_per_epoch =len(train_data_10_percent),
                 validation_data = test_data,
                 validation_steps = len(test_data),
                 callbacks = [create_tensorboard_callback(dir_name="tensorflow_hub",
                                                          experiment_name ="resnet50V2")])

Saving TensorBoard log files to : tensorflow_hub/resnet50V2/20220218-071548
Epoch 1/5
24/24 [==============================] - 474s 20s/step - loss: 0.3149 - accuracy: 0.9387 - val_loss: 0.6619 - val_accuracy: 0.7764
Epoch 2/5
24/24 [==============================] - 495s 21s/step - loss: 0.2637 - accuracy: 0.9480 - val_loss: 0.6475 - val_accuracy: 0.7748
Epoch 3/5
24/24 [==============================] - 482s 21s/step - loss: 0.2188 - accuracy: 0.9707 - val_loss: 0.6368 - val_accuracy: 0.7828
Epoch 4/5
20/24 [========================>.....] - ETA: 16s - loss: 0.1884 - accuracy: 0.9823

KeyboardInterrupt: ignored

Our transfer learning feature extractor model outperformed all of the previous models we built by hand. We have achieved around ~91% accuracy!! with only 10% of the data

In [ ]:
# let's create a function to plot our loss curves
# we can put a function like this into a script called "helper.py and import "
import matplotlib.pyplot as plt
plt.style.use('dark_background') # set dark background for plots

def plot_loss_curves(history):
  """
  Returns training and validation metrics

  Args: 
    history : TensorFlow History object.

  Returns:
    plots of training/validation loss and accuracy metrics

  """
  
  loss = history.history["loss"]
  val_loss = history.history["val_loss"] 

  accuracy = history.history["accuracy"]
  val_accuracy = history.history["val_accuracy"]

  epochs = range(len(history.history["loss"]))

  # Plot loss
  plt.plot(epochs, loss, label = "training_loss")
  plt.plot(epochs, val_loss, label = "validation_loss")
  plt.title("Loss")
  plt.xlabel("Epochs")
  plt.legend();

  # Plot Accuracy
  plt.plot(epochs, accuracy, label ="training_accuracy")
  plt.plot(epochs, val_accuracy, label = "validation_accuracy")
  plt.title("Accuracy")
  plt.xlabel("Accuracy")
  plt.legend();

In [ ]:
plot_loss_curves(history